In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas import HDFStore
store = HDFStore("RGA_RunData.h5")
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default="plotly_white"
pio.renderers.default = 'jupyterlab'  # Alternate: 'browser' opens new browser window with plots.


In [2]:
df = [None, None, None]
df[0] = store['/period_0/All_Runs']
df[1] = store['/period_1/All_Runs']
df[2] = store['/period_2/All_Runs']

In [7]:
runs = df[1].copy()
runs.drop(runs.loc[ runs.data_start.isnull() ].index, inplace=True)
runs.drop(runs.loc[ runs.start_time.isnull() ].index, inplace=True)
runs.drop(runs.loc[ runs.end_time.isnull() ].index, inplace=True)
runs["center"] = runs.start_time + (runs.end_time - runs.start_time)/2
runs["dt"] = (runs.end_time - runs.start_time).map(lambda x: x.total_seconds()*1000)
runs["hover"] = runs.index.map(lambda x: f"Run: {x}, target: {runs.loc[x,'target']}")
runs.drop(runs.loc[ (runs.dt <= 0) ].index, inplace=True)
runs["event_rate"] = runs.event_count/runs.dt
runs.loc[:,["events_rate", "event_rate"]]

,events_rate,event_rate
number,,
4764,None,1.372716
4765,None,0.576
4779,None,1.05022
4780,None,0.958069
4781,None,1.748669
...,...,...
5415,2069.47,18.480528
5416,2144.47,17.575261
5417,15167.5,16.345007


In [19]:
fig = make_subplots(rows=2, specs=[[{"secondary_y": True}],[{}]])
fig.update_layout(height=900, width=1200,
                  title=go.layout.Title(text=f"Charge comparison for RGA", xanchor="left", x=0.4),
                  titlefont=dict(size=24),)
fig.add_trace(go.Bar(x=runs.center, y=runs.event_rate, width=runs.dt, name="Event rate", hovertext=runs.hover, marker=dict(color='rgba(200,200,250,0.2)') ), row=1, col=1, secondary_y=True)
fig.add_trace(go.Scatter(x=runs.center, y=1E9*runs.Fcup_charge/runs.dt, name="Uncorrected Faraday Cup Current", hovertext=runs.hover, mode="markers", marker=dict(color="yellow")), row=1, col=1)
fig.add_trace(go.Scatter(x=runs.center, y=1E9*runs.Fcup_charge_corr/runs.dt, name="Corrected Faraday Cup Current", hovertext=runs.hover, mode="markers", marker=dict(color="red")), row=1, col=1)
fig.add_trace(go.Scatter(x=runs.center, y=1E9*runs.data_charge/runs.dt, name="Current from data", mode="markers", hovertext=runs.hover, marker=dict(color="green")), row=1, col=1)

diff_corr = 100*( (runs.data_charge - runs.Fcup_charge_corr)/runs.data_charge )
diff_uncorr = 100*( (runs.data_charge - runs.Fcup_charge)/runs.data_charge )
fig.add_trace(go.Scatter(x=runs.center, y=diff_uncorr, name="Delta data - FCup uncorr",
                         mode="markers", hovertext=runs.hover, marker=dict(color="yellow")), row=2, col=1)

fig.add_trace(go.Scatter(x=runs.center, y=diff_corr, name="Delta data - FCup corrected",
                         mode="markers", hovertext=runs.hover, marker=dict(color="red")), row=2, col=1)


fig.update_yaxes(title_text="<b>Current nA</b>", titlefont=dict(size=22), secondary_y=False, tickfont=dict(size=16), row=1, col=1)
fig.update_yaxes(title_text="<b>Event rate kHz</b>", titlefont=dict(size=22, color='rgba(200,200,250,0.5)'), secondary_y=True, tickfont=dict(size=16),range = [0, 15.], row=1, col=1)
fig.update_yaxes(title_text="<b>Δ data - Fcup %</b>", titlefont=dict(size=22), secondary_y=False, tickfont=dict(size=16),range = [-20., 20.], row=2, col=1)
fig.write_html("RGA_charge.html")
fig.write_image("RGA_charge.pdf", width=2048, height=900)
fig.show()